<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/Copy_of_FineTuning_T2SQL_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 , L4  IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 or L4 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

!pip install mistral_inference -q

!pip install trl==0.8.6 -q


!pip install torch-geometric -q
!pip install sqlparse networkx -q

!pip install bitsandbytes -q

In [2]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

#print(access_token_write)

login(
  token=access_token_write,
  add_to_git_credential=True
)

Mounted at /content/gdrive
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
     pipeline,
)
from trl import SFTTrainer

In [4]:
# set device
device = 'cuda'

In [5]:
torch.__version__

'2.3.1+cu121'

In [6]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Mon Jul 15 10:37:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   41C    P8       

MISTRAL

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format


from huggingface_hub import login


login(
  token=access_token_write,
  add_to_git_credential=True
)

print()

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.3" #24 JUNE 2024

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
mistral_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
mistral_model, tokenizer = setup_chat_format(mistral_model, tokenizer)

GNN #0

In [8]:
# 3. Load and Prepare Data
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle(seed=42).select(range(125))

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [23]:
dataset

Dataset({
    features: ['answer', 'question', 'context'],
    num_rows: 12500
})

In [19]:
import colab_env
import os
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")
from huggingface_hub import login
#print(access_token_write)
login(
 token=access_token_write,
 add_to_git_credential=True
)

import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
 AutoModelForCausalLM,
 AutoTokenizer,
 BitsAndBytesConfig,
 AutoTokenizer,
 TrainingArguments,
 pipeline,
)
from trl import SFTTrainer

# set device
device = 'cuda'


tokenizer.pad_token_id = tokenizer.unk_token_id
# # set chat template to OAI chatML, remove if you start from a fine-tuned model
mistral_model, tokenizer = setup_chat_format(mistral_model, tokenizer)

# GNN
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from torch_geometric.data import Data, Batch # Import Batch here
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool # Import global_mean_pool here
import torch.optim as optim
from tqdm.auto import tqdm
import evaluate
import numpy as np

# 1. Graph Construction
import torch
import sqlparse
import networkx as nx
from torch_geometric.data import Data
from tqdm import tqdm

def sql_to_graph(question, schema, answer):
    try:
        # 1. Parse SQL Query (with robust error handling)
        try:
            statements = sqlparse.parse(question)
            print(f"Parsed statements: {statements}")
        except sqlparse.exceptions.SQLParseError as e:
            # Attempt to fix common parsing issues
            fixed_question = question.replace(';', '')
            try:
                statements = sqlparse.parse(fixed_question)
                print(f"Warning: SQL parsing initially failed for '{question}', but succeeded after removing semicolon(s).")
            except sqlparse.exceptions.SQLParseError as fixed_e:
                print(f"Warning: SQL parsing failed for '{question}' (even after fix attempt): {fixed_e}. Skipping example.")
                return None
        except ValueError as e:  # Catch potential errors in the context dict
            print(f"Warning: Context dict Value Error: '{question}': {e}. Skipping example.")
            return None
        except Exception as e:  # Catch any other unexpected parsing errors
            print(f"Unexpected error while parsing '{question}': {e}. Skipping example.")
            return None

        # 2. Create Graph
        G = nx.DiGraph()
        current_table = None
        for stmt in statements:
            if stmt.get_type() == "SELECT":
                select_items = [token for token in stmt if isinstance(token, sqlparse.sql.IdentifierList)]
                for item in select_items:
                    for token in item.get_identifiers():
                        G.add_node(token.value)  # Add column name as node (for SELECT)

            for token in stmt.flatten():
                if token.ttype is sqlparse.tokens.Keyword:
                    G.add_node(token.value.upper())
                    if token.value.upper() in ["FROM", "JOIN"]:
                        # Handle multi-table queries or subqueries
                        try:
                            current_table = next(token.flatten()).value
                        except StopIteration:
                            print(
                                f"Warning: No table found after 'FROM' or 'JOIN' in '{question}'. Skipping example."
                            )
                            return None
                elif token.ttype in (sqlparse.tokens.Name, sqlparse.tokens.Wildcard):
                    G.add_node(token.value)
                    if current_table:
                        G.add_edge(current_table, token.value)

        # Check if graph is empty
        if G.number_of_nodes() == 0 or G.number_of_edges() == 0:
            print(f"Warning: Empty graph generated for: {question}")
            return None

        # 3. Extract Node Features
        node_features = []
        for node in G.nodes():
            feature = [
                1 if node in ["SELECT", "FROM", "WHERE", "GROUP BY", "ORDER BY", "HAVING", "LIMIT", "JOIN"] else 0,  # Keyword feature
                1 if node in schema.get("table_names_original", []) else 0,  # Table feature
                1 if node in schema.get("column_names_original", []) else 0,  # Column feature
                1 if node.upper() in ["COUNT", "SUM", "AVG", "MAX", "MIN"] else 0,  # Aggregate function feature
            ]
            node_features.append(feature)

        # 4. Extract Edge Features (simple for now)
        edge_features = [[1]] * len(G.edges())

        # 5. Convert to PyTorch Geometric Data
        try:
            x = torch.tensor(node_features, dtype=torch.float)
            edge_index = torch.tensor(list(G.edges())).t().contiguous()

            # Tokenize answer, handling potential errors (KeyError for empty answers)
            try:
                answer_tokens = tokenizer(answer, return_tensors="pt")["input_ids"]
            except KeyError as e:
                print(f"Warning: Empty or untokenizable answer for '{question}': {e}. Skipping example.")
                return None
            y = answer_tokens

            # Add prints for intermediate values in `sql_to_graph`
            print(f"Graph Nodes: {G.nodes()}")
            print(f"Graph Edges: {G.edges()}")
            print(f"Node Features:\n{x}")
            print(f"Edge Index:\n{edge_index}")
            print(f"Answer Tokens: {y}")

            if x.size(0) == 0 or edge_index.size(0) == 0 or y.size(0) == 0:
                raise ValueError("Empty graph, features, or answer tokens.")

            return Data(x=x, edge_index=edge_index, y=y)

        except Exception as e:
            print(f"Error converting to PyG Data for '{question}': {e}")
            return None

    except Exception as e:
        print(f"Unexpected error in sql_to_graph for '{question}': {e}. Skipping example.")
        return None  # Return None on any unexpected error


# 1. Placeholder Conversion Function
import torch
from torch_geometric.data import Data
from tqdm import tqdm
import sqlparse

def is_valid_schema(context):
    # Since the dataset already has the schema in the correct format, no need for conversion
    return "table_names_original" in context and "column_names_original" in context

import torch
from torch_geometric.data import Data
from tqdm import tqdm

def is_valid_schema(context):
    return "table_names_original" in context and "column_names_original" in context

def convert_to_gnn(dataset):
    dataset_gnn = []
    failed_queries = []
    valid_query_count = 0

    with tqdm(total=len(dataset), desc="Converting to GNN") as loop:
        for i, example in enumerate(dataset):
            loop.set_description(f"Converting to GNN ({int(100 * i / len(dataset))}%)")

            question = example["question"]
            context = example["context"]
            answer = example["answer"]


            #print(f"\nProcessing example {i}: {question}")
            #print(f"With context: {context}")

            # Check if the context contains valid schema information
            if not is_valid_schema(context):
                #print(f"Skipping example {i} (Invalid schema): '{question}'")
                failed_queries.append(question)
                loop.update(1)
                continue

            try:
                table_names = context.get("table_names_original", [])
                column_names = context.get("column_names_original", [])
            except AttributeError as e:  # Catch if context is not a dictionary
                print(f"Warning: Context not a dictionary in example {i}: '{question}'. Skipping example.")
                failed_queries.append(question)
                loop.update(1)
                continue
            except Exception as e: # Check for invalid data
                print(f"Warning: Unexpected error in schema: '{question}': {e}. Skipping example.")
                failed_queries.append(question)
                loop.update(1)
                continue

            if not table_names or not column_names:  # Check for missing keys
                print(f"Skipping example {i} (Missing table or column names): '{question}'")
                failed_queries.append(question)
                loop.update(1)
                continue

            if not all(isinstance(name, str) for name in table_names) or not all(isinstance(name, str) for name in column_names):
                # Check if all elements in lists are strings
                print(f"Skipping example {i} (Invalid table/column name type): '{question}'")
                failed_queries.append(question)
                loop.update(1)
                continue


            try:
                graph = sql_to_graph(question, context, answer)
                if graph is None:
                    print(f"Warning: sql_to_graph returned None for example {i}: '{question}'. Skipping example.")
                    print(f"Context for failed query: {context}")   # Print the context here

                    failed_queries.append(question)
                elif graph.num_nodes == 0 or graph.num_edges == 0:
                    print(f"Warning: Empty graph generated for example {i}: '{question}'. Schema: {context}")
                    failed_queries.append(question)
                else:
                    print(f"Successfully generated graph for example {i}")
                    dataset_gnn.append(graph)
                    valid_query_count += 1
            except Exception as e:
                #print(f"Error processing example {i}: '{question}': {e}")
                #print(f"Schema for the failed query: {context}")
                failed_queries.append(question)

            loop.update(1)  # Update progress bar after each example

    # Final summary and check
    print(f"\nTotal valid queries processed: {valid_query_count}")
    print(f"Total invalid queries: {len(failed_queries)}")


    if failed_queries:
        print("\nFirst 10 failed queries and their schemas:")
        for query, ctx in zip(failed_queries[:10], [dataset[i]["context"] for i in range(len(failed_queries))[:10]]):
            print(f"- Query: {query}")
            print(f"  Schema: {ctx}\n")

    return dataset_gnn



class SQLGraphDataset(Dataset):
 def __init__(self, data):
  self.data = data
 def __len__(self):
  return len(self.data)
 def __getitem__(self, index):
  entry = self.data[index]
  question = entry["question"]
  schema = entry["context"]
  answer = entry["answer"]
  return sql_to_graph(question, schema, answer)


# 2. GNN Model
from torch_geometric.nn import GATConv
class SQLGNN(torch.nn.Module): # Inherit from torch.nn.Module
 def __init__(self, input_dim, hidden_dim, output_dim, heads=8):
  super(SQLGNN, self).__init__() # Call superclass constructor
  self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0.6)
  self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=0.6)

 def forward(self, data):
  x, edge_index, batch = data.x.to(device), data.edge_index.to(device), data.batch.to(device)  # Move graph data to GPU
  x = F.elu(self.conv1(x, edge_index))
  x = F.dropout(x, p=0.6, training=self.training)
  x = self.conv2(x, edge_index)
  x = global_mean_pool(x, batch)  # Global Mean Pooling for graph-level representation
  return x

 # Add this method to explicitly define saving state dictionary
 def save_state_dict(self, filepath):
    torch.save(super().state_dict(), filepath)


# Define a custom collate function to handle batching of graphs and text data

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Filter out 'None' values from the batch
    batch = [data for data in batch if data is not None]

    if not batch:  # If the batch is empty after filtering
        print("Warning: Empty batch encountered. Skipping this batch.")
        return None, None, None, None

    graphs, questions, schemas, answers = [], [], [], []

    for i, data in enumerate(batch):
        if data.edge_index.numel() > 0:  # Check if the graph has edges
            graphs.append(data)
            questions.append(dataset[i]["question"])
            schemas.append(dataset[i]["context"])

            # Handle different answer formats
            if isinstance(data.y, torch.Tensor):
                answers.append(data.y.tolist())
            else:
                answers.append(data.y)

    if not graphs:  # Handle case where all graphs in the batch are empty
        print("Warning: All graphs in the batch are empty. Skipping this batch.")
        return None, None, None, None

    # Pad edge_index to the maximum length in the batch
    max_edges = max([data.num_edges for data in graphs])
    for data in graphs:
        padding = max_edges - data.num_edges
        if padding > 0:
            pad = torch.zeros(2, padding, dtype=torch.long)
            data.edge_index = torch.cat([data.edge_index, pad], dim=1)

    # Pad node features (x) to the maximum number of nodes
    max_nodes = max([data.num_nodes for data in graphs])
    for data in graphs:
        padding = max_nodes - data.num_nodes
        if padding > 0:
            pad = torch.zeros(padding, data.x.size(1), dtype=torch.float)  # Pad with zeros of the same feature dimension
            data.x = torch.cat([data.x, pad], dim=0)

    return Batch.from_data_list(graphs), questions, schemas, answers





# 3. Load and Prepare Data
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle(seed=42).select(range(12500))

# Print a sample of the dataset to check if the schema and tables are in the correct format
print('\n\n')
print('Print a sample of the dataset to check if the schema and tables are in the correct format')
print(dataset[:2])
print('\n\n')


# Split dataset
train_size = int(0.8 * len(dataset))
print(f'train size: {train_size}')

train_dataset = dataset[:train_size]
val_dataset = dataset[train_size:]

#print(len(train_dataset))
#print(len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, collate_fn=collate_fn)


# Convert to GNN format
dataset_gnn = convert_to_gnn(dataset)

print(f"Number of successful graphs: {len(dataset_gnn)}")
print(f"Dataset examples are all strings: {all(isinstance(example['question'], str) for example in dataset)}")

# Analyze the Results
print(f"Total number of examples: {len(dataset)}")
#print(f"Number of failed queries: {len(failed_queries)}")



import warnings
# Check if dataset_gnn has at least one valid graph and the split will work
if not dataset_gnn:
    warnings.warn("No valid graphs were generated. Check the 'sql_to_graph' function and the dataset for errors.", UserWarning)
else:
    #4. Initialize Model, Loss, and Optimizer
    print(f'Number of successful graphs; {len(dataset_gnn)}')
    input_dim = dataset_gnn[0].num_node_features
    hidden_dim = 64
    output_dim = 128 # Assuming a generation task for simplicity
    model = SQLGNN(input_dim, hidden_dim, output_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Length of train_dataset: {len(train_loader)}")
print(f"Length of val_dataset: {len(val_loader)}")

# For generation:
criterion = torch.nn.CrossEntropyLoss(ignore_index=0) # Ignore padding in loss calculation
# For classification:
# criterion = torch.nn.BCEWithLogitsLoss() # Or other suitable loss

# 5. Training and Evaluation Functions
bleu = evaluate.load("bleu")
tokenizer.padding_side = 'left'
import torch.nn.functional as F  # Import F for the loss function


def train(model, mistral_model, loader, optimizer, mistral_optimizer, epoch, num_epochs):
    model.train()
    mistral_model.train()
    total_loss = 0

    #loop = tqdm(loader, total=len(loader), desc=f"Epoch {epoch+1}/{num_epochs}")
    #for data, questions, schemas, answers in loop:
    total_loss = 0
    loop = tqdm(loader, total=len(loader), desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch_data in loop:
        if batch_data is None:  # Check if batch is None
            continue  # Skip the batch if it's None

        data, questions, schemas, answers = batch_data  # Unpack the batch

        # Initialize graph_embeddings with an empty list
        graph_embeddings = []

        # Check if the graph data is None
        if data is not None:
            data = data.to(device)
            graph_embeddings = model(data)  # Get embeddings from GNN

        # Filter out None values from graph_embeddings
        valid_graph_embeddings = [g for g in graph_embeddings if g is not None]

        optimizer.zero_grad()
        mistral_optimizer.zero_grad()

        # Initialize mistral_inputs with an empty list
        mistral_inputs = []

        # Prepare Mistral Input
        if valid_graph_embeddings:
          mistral_inputs = [f"Question: {q}\nSchema: {s}\nGraph Embedding: {g}" for q, s, g in zip(questions, schemas, valid_graph_embeddings)]

          # Check if mistral_inputs is empty
          if mistral_inputs:
            # Tokenize and generate SQL using Mistral, increase max length
             tokenized_inputs = tokenizer(mistral_inputs, return_tensors="pt", padding=True, truncation=True, max_length=512)

          # Check for empty tokenized inputs
          for i, input_ids in enumerate(tokenized_inputs['input_ids']):
              if input_ids.numel() == 0:
                  print(f"Warning: Input example {i} has no tokens after encoding.")
                  # Handle empty example (skip or replace)

          tokenized_inputs = tokenized_inputs.to(device)

          # Set padding side to left before tokenizing
          tokenizer.padding_side = 'left'

          # Get logits for loss calculation (instead of generating)
          outputs = mistral_model(**tokenized_inputs, labels=tokenized_inputs["input_ids"])
          loss = outputs.loss
          total_loss += loss.item()

          # Backpropagation
          loss.backward()
          optimizer.step()
          mistral_optimizer.step()

          loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    %cd /content/
    #Save GNN model - UNCOMMENT THESE LINES
    model.save_state_dict(f"gnn_model_epoch_{epoch+1}.pth")
    print(f"GNN Model saved to gnn_model_epoch_{epoch+1}.pth")



Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful



Print a sample of the dataset to check if the schema and tables are in the correct format
{'answer': ['SELECT venue FROM table_name_50 WHERE away_team = "essendon"', 'SELECT MIN(game) FROM table_name_61 WHERE opponent = "phoenix" AND record = "29-17"'], 'question': ['When Essendon played away; where did they play?', 'What is the lowest numbered game against Phoenix with a record of 29-17?'], 'context': ['CREATE TABLE table_name_50 (venue VARCHAR, away_team VARCHAR)', 'CREATE TABLE table_name_61 (game INTEGER, opponent VARCHAR, record VARCHAR)']}



train size: 10000


Converting to GNN (99%): 100%|██████████| 12500/12500 [00:19<00:00, 642.92it/s]



Total valid queries processed: 0
Total invalid queries: 12500

First 10 failed queries and their schemas:
- Query: When Essendon played away; where did they play?
  Schema: CREATE TABLE table_name_50 (venue VARCHAR, away_team VARCHAR)

- Query: What is the lowest numbered game against Phoenix with a record of 29-17?
  Schema: CREATE TABLE table_name_61 (game INTEGER, opponent VARCHAR, record VARCHAR)

- Query: Who did the Texan's play on Week 4?
  Schema: CREATE TABLE table_name_37 (opponent VARCHAR, week VARCHAR)

- Query: Which Points have Touchdowns larger than 0, and an Extra points smaller than 0?
  Schema: CREATE TABLE table_name_70 (points INTEGER, touchdowns VARCHAR, extra_points VARCHAR)

- Query: What is the name of the player who is Sco and moving to greenock morton in the summer?
  Schema: CREATE TABLE table_name_83 (name VARCHAR, moving_to VARCHAR, nat VARCHAR, transfer_window VARCHAR)

- Query: Of all the contestants who got voted, what is the contestant number and name 

<ipython-input-19-5f0913dbebbe>:391: UserWarning: No valid graphs were generated. Check the 'sql_to_graph' function and the dataset for errors.
  warnings.warn("No valid graphs were generated. Check the 'sql_to_graph' function and the dataset for errors.", UserWarning)


In [17]:
len(train_dataset)

3

In [18]:
len(train_loader)

1

In [14]:
len(val_loader)

1

GNN #1

In [49]:
question=train_loader.dataset['question'][20]
print(f'Question: {question}')

Question: Who was the runner-up in the Memorial Tournament?


In [50]:
schema=train_loader.dataset['context'][20]
print(f'Schema: {schema}')

Schema: CREATE TABLE table_1602858_1 (runner_s__up VARCHAR, tournament VARCHAR)


In [51]:
query=train_loader.dataset['answer'][20]
print(f'Query: {query}')

Query: SELECT runner_s__up FROM table_1602858_1 WHERE tournament = "Memorial tournament"


In [ ]:
  # 5. Training Loop (Main Refinement)
num_epochs = 10
for epoch in range(num_epochs):
    # Check if the training loader is empty
    if len(train_loader) == 0:
        print("Warning: Training loader is empty. Check your dataset and dataloader setup.")
        break  # Exit the training loop

    train(model, mistral_model, train_loader, optimizer, mistral_optimizer, epoch, num_epochs)

#Save GNN model - UNCOMMENT THESE LINES
%cd /content/
model.save_state_dict(f"gnn_model_epoch_{epoch+1}.pth")
print(f"GNN Model saved to gnn_model_epoch_{epoch+1}.pth")

In [ ]:
mistral_optimizer = optim.Adam(mistral_model.parameters(), lr=5e-5)

# Training Loop
num_epochs = 1  # Or your desired number of epochs
for epoch in tqdm(range(num_epochs), desc="Overall Training Progress"):
    train(model, mistral_model, train_loader, optimizer, mistral_optimizer, epoch, num_epochs)
    #evaluate(model, mistral_model, val_loader, bleu)


In [ ]:
def evaluate(model, mistral_model, loader, bleu):
    model.eval()
    mistral_model.eval()

    predictions = []
    references = []

    print(f"Number of examples in validation set: {len(loader.dataset)}")

    # Decode references only once
    for item in loader.dataset:
        if isinstance(item, dict):
            ref_tokens = item["answer"]
        elif isinstance(item, torch_geometric.data.Data):
            ref_tokens = item.y.tolist()  # Convert tensor to list
        else:
            raise ValueError(f"Unsupported data type in dataset: {type(item)}")

        # Handle potential issues with missing or invalid 'answer'
        if ref_tokens is not None:
            for token_list in ref_tokens:
                decoded_ref = tokenizer.decode(token_list, skip_special_tokens=True)
                references.append([decoded_ref])
        else:
            print("Warning: Skipping example with missing or invalid reference.")

    with torch.no_grad():
        for batch_data in tqdm(loader, desc="Evaluating"):
            if batch_data is None:
                continue

            data, questions, schemas, answers = batch_data

            graph_embeddings = []
            if data is not None:
                data = data.to(device)
                try:
                    graph_embeddings = model(data)
                except Exception as e:
                    print(f"Error generating graph embeddings: {e}")
                    continue  # Skip this batch if there's an error

            # If the model doesn't generate any embeddings, use an empty tensor
            if graph_embeddings is None or len(graph_embeddings) == 0:
                graph_embeddings = torch.tensor([])

            # Check if any graph embedding is non-zero
            if graph_embeddings.any():
                mistral_inputs = [f"Question: {q}\nSchema: {s}\nGraph Embedding: {g}"
                                  for q, s, g in zip(questions, schemas, graph_embeddings)]

                # Tokenize and generate SQL using Mistral
                tokenized_inputs = tokenizer(mistral_inputs, return_tensors="pt", padding=True, truncation=True).to(device)
                output_sequences = mistral_model.generate(**tokenized_inputs, max_new_tokens=128)
                generated_sql = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

                predictions.extend(generated_sql)

                # Print generated and reference SQL (for debugging purposes)
                print(f"Generated SQL: {generated_sql}")
                print(f"Reference SQL: {answers}")

    # Ensure both lists have the same length
    min_len = min(len(predictions), len(references))
    predictions = predictions[:min_len]
    references = references[:min_len]

    # ... (rest of the evaluate function)

    if predictions and references:
        try:
            bleu_scores = bleu.compute(predictions=predictions, references=references)  # Get the dictionary of BLEU scores
            bleu_score = bleu_scores["bleu"]  # Extract the overall BLEU score
            print(f"BLEU Score: {bleu_score:.4f}")
        except ZeroDivisionError:
            print("ZeroDivisionError during BLEU calculation. Likely no matching n-grams.")
    else:
        print("No valid predictions or references found for BLEU calculation.")



In [ ]:
%cd /content/
model = SQLGNN(input_dim, hidden_dim, output_dim)  # Recreate the model architecture
model.load_state_dict(torch.load(f"/content/gnn_model_epoch_{epoch+1}.pth"))  # Load the saved parameters
model.to(device)  # Move the model to the desired device (e.g., GPU)

In [ ]:
# Final Evaluation (using the best model)
evaluate(model, mistral_model, val_loader, bleu)